# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [27]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%pip install dotenv

In [28]:
import os
from google.colab import userdata

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')
#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
#HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
#!pip install pandas

In [29]:
import pandas as pd
df = pd.read_csv('Data.csv')
#df = pd.read_csv('./data/Data.csv')

In [30]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
!pip install langchain_community
!pip install langchain_openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00


In [31]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [32]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)


In [33]:
prompt = ChatPromptTemplate.from_template(template="Please write a compelling product description for the following product: {product_description}")

In [34]:
chain = LLMChain(llm=llm, prompt=prompt)

In [35]:
product = "Foldable bicycle"#Select a product type to be describe
chain.run(product)

"Introducing the ultimate in convenience and practicality - the foldable bicycle! Say goodbye to the hassle of storing and transporting a traditional bike, and hello to effortless portability with this sleek and compact design. Whether you're a city commuter looking for an easy way to navigate through traffic, or an adventurous traveler wanting to explore new landscapes, the foldable bicycle is the perfect solution for on-the-go cycling. With its durable construction and innovative folding mechanism, this bike offers the same reliability and performance as a traditional bicycle, but with the added bonus of being able to fold up in seconds for easy storage in your car, office, or apartment. Get ready to revolutionize your ride with the foldable bicycle - the future of cycling is here!"

## SimpleSequentialChain

In [36]:
from langchain.chains import SimpleSequentialChain

In [46]:
llm = ChatOpenAI(temperature=0.9,model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(template="Please write a compelling product description for the following product: {product_description}"
    #Repeat the initial query or create a new query that would feed into the second prompt
                                                )

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [47]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
    template="Please write the types of following product: {product}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [48]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)

In [49]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Introducing the ultimate solution for urban commuters and cycling enthusiasts alike - the foldable bicycle! Say goodbye to bulky and cumbersome bikes that are difficult to transport and store. Our foldable bicycle is lightweight, easy to fold, and compact, making it perfect for commuters who need a convenient and versatile mode of transportation.

Whether you're heading to work, running errands, or exploring the city, this foldable bicycle is your reliable companion. With its sturdy frame and high-quality components, you can ride with confidence knowing you have a reliable and efficient bike by your side.

Not only is our foldable bicycle practical, but it's also stylish and sleek, turning heads wherever you go. Stand out from the crowd and make a statement with this modern and innovative mode of transportation.

Don't compromise on convenience or style - invest in a foldable bicycle and experience the freedom of easy and efficient cyclin

'1. Foldable bicycle\n2. Urban commuter bicycle\n3. Cycling enthusiast bicycle\n4. Lightweight bicycle\n5. Compact bicycle\n6. Convenient transportation solution\n7. Versatile mode of transportation\n8. Sturdy frame bicycle\n9. High-quality components bicycle\n10. Stylish and sleek bicycle\n11. Modern and innovative transportation option\n12. Efficient cycling companion\n13. Practical and stylish foldable bicycle'

**Repeat the above twice for different products**

## SequentialChain

In [50]:
from langchain.chains import SequentialChain

In [63]:
llm = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  template="Translate English to French: {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="French_Review" #Give a name to your output
                    )


In [64]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    template="Summarize the following review in one sentence: {review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary" #give a name to this output
                    )


In [79]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    template="Translate to English: {review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="eng_text"
                      )


In [80]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        template="Write a follow up response to the following summary in the specified language: \n\nSummary: {summary}\n\nLanguage: {eng_text}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [81]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["summary", "eng_text", "followup_message"],
    verbose=True
)

In [82]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'summary': "Le reviewer trouve le goût médiocre, la mousse ne tient pas et pense que c'est un vieux lot ou une contrefaçon.",
 'eng_text': "I find the taste to be mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better...Old batch or counterfeit!?",
 'followup_message': "Je suis désolé d'entendre que votre expérience avec ce produit n'a pas été satisfaisante. Il est possible que vous ayez reçu un lot défectueux ou contrefait. Je vous recommande de contacter le fabricant pour signaler le problème et peut-être demander un remboursement ou un remplacement. J'espère que votre prochaine expérience avec ce produit sera meilleure."}

**Repeat the above twice for different products or reviews**

## Router Chain

In [83]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [84]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [85]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [87]:
llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

In [88]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [89]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [90]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [91]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [92]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [93]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [94]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for the compliment! The answer to the question "what is 2 + 2" is 4.'

In [95]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information that can be accessed and utilized by cells as needed. This allows for the regulation of gene expression, the repair of damaged DNA, and the adaptation of cells to changing environmental conditions.\n\nIn summary, every cell in our body contains DNA because it is essential fo

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [97]:
chain.run("Describe the major events of the French Revolution.")
chain.run("Write a Python function to calculate the factorial of a number.")




> Entering new MultiPromptChain chain...
History: {'input': 'Describe the major events of the French Revolution.'}
> Finished chain.


> Entering new MultiPromptChain chain...
computer science: {'input': 'Write a Python function to calculate the factorial of a number.'}
> Finished chain.


'Sure! Here is a Python function to calculate the factorial of a number:\n\n```python\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\n# Test the function\nnum = 5\nprint(f"The factorial of {num} is {factorial(num)}")\n```\n\nThis function uses recursion to calculate the factorial of a number. It checks if the input number is 0, in which case it returns 1. Otherwise, it multiplies the number by the factorial of (n-1) until it reaches 0.'